# Flood segmentation

In [1]:
from fastai.vision.all import *
import torch

# Check if GPU is available and enable it
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set up paths
working_path = Path.cwd()
input_path = Path('/kaggle/input')

# Create necessary directories
folders = ('train', 'valid')
labels = ('0', '1')

for folder in folders:
    if not (working_path/folder).exists():
        (working_path/folder).mkdir()
    for label in labels:
        if not (working_path/folder/label).exists():
            (working_path/folder/label).mkdir()

# Get image paths
train_image_paths = sorted(input_path.rglob('train/*.png'))
valid_image_paths = sorted(input_path.rglob('test/*.png'))
print(f"Found {len(train_image_paths)} training images and {len(valid_image_paths)} validation images")

# Organize images into folders based on their labels
try:
    for image_path in train_image_paths:
        label = '1' if '_1' in image_path.stem else '0'
        with (working_path/'train'/label/image_path.name).open(mode='xb') as f:
            f.write(image_path.read_bytes())
except FileExistsError:
    print("Training images have already been moved.")
else:
    print("Training images moved.")

try:
    for image_path in valid_image_paths:
        label = '1' if '_1' in image_path.stem else '0'
        with (working_path/'valid'/label/image_path.name).open(mode='xb') as f:
            f.write(image_path.read_bytes())
except FileExistsError:
    print("Testing images have already been moved.")
else:
    print("Testing images moved.")

# Create DataBlock with augmentations
augmented_dataloaders = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=GrandparentSplitter(),
    get_y=parent_label,
    item_tfms=RandomResizedCrop(192, min_scale=0.5),
    batch_tfms=aug_transforms()
).dataloaders(working_path, bs=32)

# Create learner with GPU support
augmented_learner = vision_learner(augmented_dataloaders, resnet18, metrics=error_rate)

# Add model saving callback
augmented_learner.add_cb(SaveModelCallback(monitor='error_rate', fname='best_model'))

# Train model
augmented_learner.fine_tune(9)

# Save final model
augmented_learner.save('final_model')

# Function to infer on new images
def infer_image(image_path):
    img = PILImage.create(image_path)
    label, _, probabilities = augmented_learner.predict(img)
    
    if label == '0':
        result = f"Not flooded with probability {probabilities[0]*100:.2f}%"
    else:
        result = f"Flooded with probability {probabilities[1]*100:.2f}%"
    
    return img, result

# Test on a few images
test_images = sorted((input_path/'floodclassifiertestset').rglob('*.*'))
for i, img_path in enumerate(test_images[:5]):
    img, result = infer_image(img_path)
    print(f"Image {i+1}: {result}")
    # Uncomment to display images if in a notebook environment
    # display(img)

# Function to load the best model for future use
def load_best_model():
    learner = vision_learner(augmented_dataloaders, resnet18, metrics=error_rate)
    learner.load('best_model')
    return learner

# Example of loading best model
# best_model = load_best_model()

Using device: cuda
Found 270 training images and 52 validation images
Training images moved.
Testing images moved.


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time
0,0.917894,0.904248,0.326923,00:07


Better model found at epoch 0 with error_rate value: 0.32692307233810425.


epoch,train_loss,valid_loss,error_rate,time
0,0.508307,0.386174,0.211538,00:02
1,0.408248,0.189076,0.076923,00:01
2,0.357820,0.204755,0.057692,00:01
3,0.313139,0.247631,0.076923,00:02
4,0.287985,0.118160,0.038462,00:01
5,0.252988,0.107949,0.038462,00:01
6,0.215450,0.082234,0.038462,00:01
7,0.182943,0.069889,0.057692,00:01
8,0.159153,0.067807,0.038462,00:02


Better model found at epoch 0 with error_rate value: 0.21153846383094788.
Better model found at epoch 1 with error_rate value: 0.07692307978868484.
Better model found at epoch 2 with error_rate value: 0.057692307978868484.
Better model found at epoch 4 with error_rate value: 0.03846153989434242.


In [2]:
!pip list | grep fastai

fastai                                2.7.9
